#### Capstone Project
Arthur: Wilson Lau
Date: 2015 November

### Capstone Milestone Report

## Introduction
The problem I want to tackle in my capstone project is to detect investment opportunities for cash flow real estate property.  

## The Problem
Real estate investment is a passive income opportunity for investors to accumulate return without active management.  Huge long term appreciation is very much possible for real estate.  However, appreciation often happens in short period of time and the appreciation is heavily affected local market condition.  It is difficult for out-of-the-market investors to detect the trend and acquire properties before price goes up.  My capstone project is trying to help investors to detect real estate investment opportunities and be able to predict which zip code will experience long term appreciation in the near future. 

My project is divided into two stages:

# Stage One
- the first stage is to use anomaly detection technique to discover which zipcodes have success sprike of upward price appreciation. 
- The Anomaly detection is compared new monthly updated home price with historical price as well as comparing with the home price of the surrending areas based on neighborhoods, city, menpolian area, and state.

# Stage Two
- Based on the result of stage one, more data will be collected for the zipcodes that have sudden increase of home price.  
- Those new dataset will be used to figure out what feature contributes to the price appreciation and a predictive model will be created using those features. 

## Dateset
Dataset used is downloaded from Quandl's Zillow data.  This dataset is a monthly time series recording the change of home price. 

# What important fields and information does the data set have?
The important columns for this dataset is time(date end of the month), home price and zipcode. 



# What are its limitations i.e. what are some questions that you cannot answer with this data set?
This dataset actually have 22 columns.  For example, it has columns for different home size e.g. 1 bedroom, 2 bedroom, 3 bedroom, price per square foot, average listing price, etc...  Since it seems that most of these columns are highly corelate to home price, they may be very help for predicting home price. 

# What kind of cleaning and wrangling did you need to do?
Quandl's API is very powerful and very flexible.  At the same time, it is a bit complicated to pull the home price.  One way to pull home price is to pass zipcode as input parameter, and Quandl will return the historical price data with all the columns related to that zipcode.  So, to get the data, I will need to use a zipcode dataset that contains all the zipcode of the US and then use the Quandl API to pull data one zipcode at a time. 

# Are there other datasets you can find, use and combine with, to answer the questions that matter?
To order to group home price data by city, state, etc.., I will need to find other dataset that relates individual zipcodes with their city name and state name.  I will need to merge the dataset together in order to find out which zipcode belongs to which city or state. 

## Preliminary Findings and Hypothesis test.
.....






In [34]:
''' Problem: Try to find which zipcode and in which months has its price increase sigificantly higher than the mean price with
the same zipcode and also than the rest of the zipcode. '''

#load home price data into dataframe
import numpy as np
import pandas as pd
import pickle
from scipy import stats
import os
import glob
from os import path
import dateUtility
import datetime as dt

#load home price data from Datafiles
# df = pickle.load( open( ".\DataPfiles\93063_to_93524.p", "rb" ) ) 
df = pd.DataFrame()
for filename in glob.glob(os.path.join('.\\DataPfiles\\', '9*.p')):
    df = pd.concat([df, pd.read_pickle(filename)], axis=0)
# there may be a problem here.  There are only about 1700 zipcode in the .p file although CA has over 2700 zipcode

### Pivot indivudal columns into a new dataframe for analysis: 
### 1) Put 1) All Homes prirce 2) Price-To-Rent Ratio into rows and grouped them by zipcode
df = df.reset_index()
dfAllHomes = df.pivot(index='index', columns='ZipCode', values='All Homes')
dfPriceToRent = df.pivot(index='index', columns='ZipCode', values='Price-to-Rent Ratio')  #I don't make use of this feature now,but will do later

### calcaluate the home price's moving average for every 3 months, with minimal of 3 months
dfAllHomesMovingAve = pd.rolling_mean(dfAllHomes,window=3,min_periods=3)

### shift entire set of Moving Average number downward for one month
### This is needed to line up the previous month moving average with the current month moving average in the same row
dfAllHomesMovingAve = dfAllHomesMovingAve.shift(periods=1,freq=None,axis=0)

### since I shift the all rows for one month earlier, so I can just subtract the two dataframe and calculate the price change.
dfAllHomesDiffFromMovAve = dfAllHomes - dfAllHomesMovingAve #price change
dfAllHomesDiffFromMovAvePercent = (dfAllHomes - dfAllHomesMovingAve)/dfAllHomes.shift(periods=1,freq=None,axis=0) #price percentage change
NumOfMonthForward = 6 #set the number of month to include for calculating the predicting average percentage increase
dfAllHomesDiffFromMovAvePercentMovAve = pd.rolling_mean(dfAllHomesDiffFromMovAvePercent,window=NumOfMonthForward,min_periods=NumOfMonthForward) # calculate the moving average percentage increase for 6 months
dfAllHomesDiffFromMovAvePercentMovAve2 = dfAllHomesDiffFromMovAvePercentMovAve.shift(periods=-(NumOfMonthForward-1),freq=None,axis=0) # shift dataframe upward for 5 record so the current month shows the moving average percentage increase for the next 6 months(including current month)
dfAllHomesStdDevInZipCode = dfAllHomesDiffFromMovAvePercentMovAve2.copy() #dataframe to hold z score within zipcode

##loop through each zipcode column
for x in dfAllHomesStdDevInZipCode.columns:
    if sum(dfAllHomesStdDevInZipCode[x].isnull()) > 0:  # if any month has null value, drop the entire zipcode
        dfAllHomesStdDevInZipCode.drop(x,axis=1)
    dfAllHomesStdDevInZipCode[x] = (dfAllHomesStdDevInZipCode[x] - dfAllHomesStdDevInZipCode[x].mean()) / dfAllHomesStdDevInZipCode[x].std()  #calculate z score for All Homes price each month

##Calculate z score comparing price across zipcode
dfAllHomesStdDevAcrossZipCode = dfAllHomesDiffFromMovAvePercent.copy() #dataframe to hold z score across different zipcode
dfAllHomesStdDevAcrossZipCode['MeanHomePrice'] = dfAllHomesStdDevAcrossZipCode.mean(axis=1)
dfAllHomesStdDevAcrossZipCode['StdDevHomePrice'] = dfAllHomesStdDevAcrossZipCode.std(axis=1)
dfAllHomesStdDevAcrossZipCode = (dfAllHomesStdDevAcrossZipCode - dfAllHomesStdDevAcrossZipCode['MeanHomePrice']) / dfAllHomesStdDevAcrossZipCode['StdDevHomePrice']

##drop the two added columns so that match the dataframe size of df..InZipCode and df..AcrossZipCode
dfAllHomesStdDevAcrossZipCode = dfAllHomesStdDevAcrossZipCode.drop('MeanHomePrice',axis=1)
dfAllHomesStdDevAcrossZipCode = dfAllHomesStdDevAcrossZipCode.drop('StdDevHomePrice',axis=1)

##Abnormally Detection method ONE
##Find out which zipcode and its time frame in the dataframe has z score large than 2 in both df..InZipCode and df..AcrossZipCode dataframe
targetZscore = 2
targetZipcodeBoolean = ((dfAllHomesStdDevInZipCode > targetZscore) & (dfAllHomesStdDevAcrossZipCode > targetZscore))
# targetZipCodes = dfAllHomesMovingAve[targetZipcodeBoolean]#.dropna
selectedZipCodes = pd.DataFrame(dfAllHomesMovingAve[targetZipcodeBoolean].sum(axis = 0) > 0)
selectedZipCodes[selectedZipCodes[0] == True]

### pivot a new dataframe moving targeted zipcodes into index joining the month-end time (multi-level) index
TargetZipCode = pd.DataFrame(targetZipcodeBoolean.stack())  # stack zipcode from column names into column
TargetZipCode.columns = ['PredictZipCode'] # specify columne name
TargetZipCode = pd.DataFrame(TargetZipCode.reset_index()) # move index into column
dfPredictZipCode = pd.merge(df,TargetZipCode,on=['index','ZipCode']) #merge targeted zipcode column for prediction into main dataframe(which is loaded from 9*.p files)
dfPredictZipCode.rename(columns={'index':'Month'}, inplace=True)
dfPredictZipCode['Month'] = dfPredictZipCode['Month'].apply(lambda x: dt.datetime.strftime(x, '%Y-%m-%d'))
dfPredictZipCode.set_index(['Month','ZipCode'],inplace=True)
print sum(dfPredictZipCode.PredictZipCode)
dfPredictZipCode.head()
# dfTargetZipCode.columns = ['TargetZipCode']
# dfTargetZipCode['TargetZipCode'].sum() 

### tasks to do
### process IRS dataaset
### process Census dataset
### process proximityone dataset
### apply training model to learn how IRS and Census dataset can predict target zipcodes
### evaluate and test model by breaking down into training data and test data
### apply feature scaling to dataset
### use PCA to combine some features e.g. I think I can combine all the home price for different number of bedrooms into one feature
### modify withinZipcode detection not to use z score, but just average monthly price increase over a certain percentage
### use F1 score and some other ways to evaluate the performance of the model
### apply visualization on data; add Google maps to show the results
### rewrite Storytelling

###########process IRS dataset
### prepare 1998 IRS data
irsdata = pd.read_csv('./DataPfiles/98zp05ca.csv',header=4, na_values=['**','--'])
irsCol = ['ZipCode','NumberOfReturns','ExemptionsRtn','DepedentExemptionsAmt','AGI','SalariesWagesRtn','SalariesWagesAmt', \
          'TaxableInterestRtn','TaxableInterestAmt', \
          'TotalTaxRtn','TotalTaxAmt','ScheduleCTotalRtn','ScheduleFTotalRtn',\
          'ScheduleARtn'
          ]
# irsCol = ['ZipCode','NumberOfReturns','ExemptionsRtn','DepedentExemptionsAmt','AGI','SalariesWagesRtn','SalariesWagesAmt', \
#           'TaxableInterestRtn','TaxableInterestAmt','EarnedIncomeCreditRtn','EarnedIncomeCreditAmt', \
#           'TotalTaxRtn','TotalTaxAmt','ScheduleCTotalRtn','ScheduleCTotalScheduleC','ScheduleFTotalRtn','ScheduleFTotalScheduleF',\
#           'ScheduleARtn','ScheduleAAmt'
#           ]
irsdata.columns= irsCol #assign column names to dataframe
chars_to_remove = ['*', '**', '--',',']
for col in irsdata.columns: #clean up data by removing non numeric characters
    irsdata[col] = irsdata[col].str.translate(None, ''.join(chars_to_remove))

zipcode = pd.read_csv('./DataPfiles/us_postal_codes.csv')
calzipcode = zipcode[zipcode.State == 'California']
# print irsdata.columns
# print calzipcode.columns
# print irsdata.ZipCode
irsdata['ZipCode'] = irsdata['ZipCode'].str.strip()  #strip white space in the cell
print sum(irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str)))  #match the rows in irsdata with calzipcode
irsdata1998 = irsdata[irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str))]
irsdata1998['AverageAGI'] = irsdata1998.AGI.astype(int) #/ irsdata1998['NumberOfReturns'])
irsdata1998['NumberOfReturns'] = irsdata1998['NumberOfReturns'].astype(int)
irsdata1998['AverageAGI'] = irsdata1998['AverageAGI'] * 1000 / irsdata1998.NumberOfReturns

irsdata1998full = pd.DataFrame(columns=irsdata1998.columns.to_series().append(pd.Series("Month")))
# irsdata1998full = irsdata1998.copy()

for x in range(1,irsdata1998.shape[0]):
    listtemp = []
    irsdata1998t = pd.DataFrame(columns=irsdata1998.columns)
    irsdata1998t = irsdata1998t.append([irsdata1998.iloc[x]]*12,ignore_index=True)
    for i in range(12):
        thisMonth = ("0%i"%(i+1,))[-2:]
        d = dateUtility.mkDateTime("1998-%s-02"%thisMonth)
#        print dateUtility.mkLastOfMonth(d).strftime("%Y-%m-%d")
        listtemp.append(dateUtility.mkLastOfMonth(d).strftime("%Y-%m-%d"))
    irsdata1998t["Month"] = pd.Series(listtemp)
    irsdata1998full = irsdata1998full.append(irsdata1998t, ignore_index=True)

irsdata1998full.set_index(['Month','ZipCode'],inplace=True)  #set dataframe index to match with zipcode price dataframe
irsdata1998small = irsdata1998full[irsdata1998full.index.get_level_values('Month') == '1998-03-31']
irsdata1998small2 = irsdata1998full[irsdata1998full.index.get_level_values('ZipCode') == '90011']

# irsdata1998small = irsdata1998full[irsdata1998full.index.get_level_values('Month') == '1998-03-31']
### merge IRS data with zipcode price data
irsdatafull = irsdata1998full
print "done"
# dfPredictZipCodesmall = dfPredictZipCode[dfPredictZipCode.index.get_level_values('Month') == '1998-03-31']
# dfPredictZipCodesmall = pd.concat([dfPredictZipCodesmall.unstack(level=1),irsdata1998small.unstack(level=1)], axis=1)
# dfPredictZipCodesmall.stack(level=1)



570
2396

C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [35]:
### prepare 2001 IRS data
irsdata = pd.read_csv('./DataPfiles/01zp05ca.csv',header=4, na_values=['**','--'])
irsCol = ['ZipCode','NumberOfReturns','ExemptionsRtn','DepedentExemptionsAmt','AGI','SalariesWagesRtn','SalariesWagesAmt', \
          'TaxableInterestRtn','TaxableInterestAmt', \
          'TotalTaxRtn','TotalTaxAmt','ScheduleCTotalRtn','ScheduleFTotalRtn',\
          'ScheduleARtn'
          ]
irsdata.columns= irsCol
irsdata.head(10)
chars_to_remove = ['*', '**', '--',',']
for col in irsdata.columns:
    irsdata[col] = irsdata[col].str.translate(None, ''.join(chars_to_remove))

zipcode = pd.read_csv('./DataPfiles/us_postal_codes.csv')
calzipcode = zipcode[zipcode.State == 'California']
# print irsdata.columns
# print calzipcode.columns
# print irsdata.ZipCode
irsdata['ZipCode'] = irsdata['ZipCode'].str.strip()  #strip white space in the cell
print sum(irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str)))  #match the rows in irsdata with calzipcode
irsdata2001 = irsdata[irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str))]
irsdata2001['AverageAGI'] = irsdata2001.AGI.astype(int) #/ irsdata1998['NumberOfReturns'])
irsdata2001['NumberOfReturns'] = irsdata2001['NumberOfReturns'].astype(int)
irsdata2001['AverageAGI'] = irsdata2001['AverageAGI'] * 1000 / irsdata2001.NumberOfReturns
print irsdata2001['AverageAGI'].mean()

irsdata2001full = pd.DataFrame(columns=irsdata2001.columns.to_series().append(pd.Series("Month")))
# irsdata1998full = irsdata1998.copy()

for x in range(1,irsdata2001.shape[0]):
    listtemp = []
    irsdata2001t = pd.DataFrame(columns=irsdata2001.columns)
    irsdata2001t = irsdata2001t.append([irsdata2001.iloc[x]]*12,ignore_index=True)
    for i in range(12):
        thisMonth = ("0%i"%(i+1,))[-2:]
        d = dateUtility.mkDateTime("2001-%s-02"%thisMonth)
        listtemp.append(dateUtility.mkLastOfMonth(d).strftime("%Y-%m-%d"))
    irsdata2001t["Month"] = pd.Series(listtemp)
    irsdata2001full = irsdata2001full.append(irsdata2001t, ignore_index=True)
    
irsdata2001full.set_index(['Month','ZipCode'],inplace=True)  #set dataframe index to match with zipcode price dataframe

### merge IRS data with zipcode price data
irsdatafull = irsdatafull.append(irsdata2001full)
# dfPredictZipCodeFinal = pd.concat([dfPredictZipCode.unstack(level=1),irsdatafull.unstack(level=1)], axis=1)
# dfPredictZipCodeFinal = dfPredictZipCodeFinal.stack(level=1)

# dfPredictZipCodeFinal = pd.concat([dfPredictZipCode.unstack(level=1),irsdata1998full.unstack(level=1)], axis=1)
# dfPredictZipCodeFinal = dfPredictZipCodeFinal.stack(level=1)
# dfPredictZipCodeFinal = pd.concat([dfPredictZipCodeFinal.unstack(level=1),irsdata2001full.unstack(level=1)], axis=1)
# dfPredictZipCodeFinal = dfPredictZipCodeFinal.stack(level=1)

print "done"
# b = dfPredictZipCodeFinal.index.get_level_values(0) == '1998-09-30'  #for testing the result
# b1 = dfPredictZipCodeFinal.index.get_level_values(0) == '2001-09-30'  #for testing the result
# dfPredictZipCodeFinal[b1]  #for testing the result

C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2412
47996.9858028
done


C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [36]:
### prepare 2002 IRS data
irsdata = pd.read_csv('./DataPfiles/zptab02ca.csv',header=4, na_values=['**','--'])
irsCol = ['ZipCode','NumberOfReturns','ExemptionsRtn','DepedentExemptionsAmt','AGI','SalariesWagesRtn','SalariesWagesAmt', \
          'TaxableInterestRtn','TaxableInterestAmt', \
          'TotalTaxRtn','TotalTaxAmt','ScheduleCTotalRtn','ScheduleFTotalRtn',\
          'ScheduleARtn'
          ]
irsdata.columns= irsCol
irsdata.head(10)
chars_to_remove = ['*', '**', '--',',']
for col in irsdata.columns:
    irsdata[col] = irsdata[col].str.translate(None, ''.join(chars_to_remove))

zipcode = pd.read_csv('./DataPfiles/us_postal_codes.csv')
calzipcode = zipcode[zipcode.State == 'California']
# print irsdata.columns
# print calzipcode.columns
# print irsdata.ZipCode
irsdata['ZipCode'] = irsdata['ZipCode'].str.strip()  #strip white space in the cell
print sum(irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str)))  #match the rows in irsdata with calzipcode
irsdata2002 = irsdata[irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str))]
irsdata2002['AverageAGI'] = irsdata2002.AGI.astype(int) #/ irsdata2002['NumberOfReturns'])
irsdata2002['NumberOfReturns'] = irsdata2002['NumberOfReturns'].astype(int)
irsdata2002['AverageAGI'] = irsdata2002['AverageAGI'] * 1000 / irsdata2002.NumberOfReturns
print irsdata2002['AverageAGI'].mean()

irsdata2002full = pd.DataFrame(columns=irsdata2002.columns.to_series().append(pd.Series("Month")))

for x in range(1,irsdata2002.shape[0]):
    listtemp = []
    irsdata2002t = pd.DataFrame(columns=irsdata2002.columns)
    irsdata2002t = irsdata2002t.append([irsdata2002.iloc[x]]*12,ignore_index=True)
    for i in range(12):
        thisMonth = ("0%i"%(i+1,))[-2:]
        d = dateUtility.mkDateTime("2002-%s-02"%thisMonth)
        listtemp.append(dateUtility.mkLastOfMonth(d).strftime("%Y-%m-%d"))
    irsdata2002t["Month"] = pd.Series(listtemp)
    irsdata2002full = irsdata2002full.append(irsdata2002t, ignore_index=True)
    
irsdata2002full.set_index(['Month','ZipCode'],inplace=True)  #set dataframe index to match with zipcode price dataframe

### merge IRS data with zipcode price data
irsdatafull = irsdatafull.append(irsdata2002full)
dfPredictZipCodeFinal = pd.concat([dfPredictZipCode.unstack(level=1),irsdatafull.unstack(level=1)], axis=1)
dfPredictZipCodeFinal = dfPredictZipCodeFinal.stack(level=1)
print "done"

C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2414
49689.0262676
done


C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [39]:
### prepare 2004 IRS data
irsdata = pd.read_csv('./DataPfiles/ZIPCode2004CA.csv',header=4, na_values=['**','--'])
irsCol = ['ZipCode','NumberOfReturns','ExemptionsRtn','DepedentExemptionsAmt','AGI','SalariesWagesRtn','SalariesWagesAmt', \
          'TaxableInterestRtn','TaxableInterestAmt', \
          'TotalTaxRtn','TotalTaxAmt','ScheduleCTotalRtn','ScheduleFTotalRtn',\
          'ScheduleARtn'
          ]
irsdata.columns= irsCol
irsdata.head(10)
chars_to_remove = ['*', '**', '--',',']
for col in irsdata.columns:
    irsdata[col] = irsdata[col].str.translate(None, ''.join(chars_to_remove))

zipcode = pd.read_csv('./DataPfiles/us_postal_codes.csv')
calzipcode = zipcode[zipcode.State == 'California']
# print irsdata.columns
# print calzipcode.columns
# print irsdata.ZipCode
irsdata['ZipCode'] = irsdata['ZipCode'].str.strip()  #strip white space in the cell
print sum(irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str)))  #match the rows in irsdata with calzipcode
irsdata2004 = irsdata[irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str))]
irsdata2004['AverageAGI'] = irsdata2004.AGI.astype(int) #/ irsdata2002['NumberOfReturns'])
irsdata2004['NumberOfReturns'] = irsdata2004['NumberOfReturns'].astype(int)
irsdata2004['AverageAGI'] = irsdata2004['AverageAGI'] * 1000 / irsdata2004.NumberOfReturns
print irsdata2004['AverageAGI'].mean()

irsdata2004full = pd.DataFrame(columns=irsdata2004.columns.to_series().append(pd.Series("Month")))

for x in range(1,irsdata2004.shape[0]):
    listtemp = []
    irsdata2004t = pd.DataFrame(columns=irsdata2004.columns)
    irsdata2004t = irsdata2004t.append([irsdata2004.iloc[x]]*12,ignore_index=True)
    for i in range(12):
        thisMonth = ("0%i"%(i+1,))[-2:]
        d = dateUtility.mkDateTime("2004-%s-02"%thisMonth)
        listtemp.append(dateUtility.mkLastOfMonth(d).strftime("%Y-%m-%d"))
    irsdata2004t["Month"] = pd.Series(listtemp)
    irsdata2004full = irsdata2004full.append(irsdata2004t, ignore_index=True)
    
irsdata2004full.set_index(['Month','ZipCode'],inplace=True)  #set dataframe index to match with zipcode price dataframe

### merge IRS data with zipcode price data
irsdatafull = irsdatafull.append(irsdata2004full)
dfPredictZipCodeFinal = pd.concat([dfPredictZipCode.unstack(level=1),irsdatafull.unstack(level=1)], axis=1)
dfPredictZipCodeFinal = dfPredictZipCodeFinal.stack(level=1)
print "done"

C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2390
52308.0969728
done


C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [41]:
### prepare 2005 IRS data
irsdata = pd.read_csv('./DataPfiles/ZIPCode2005CA.csv',header=4, na_values=['**','--'])
irsCol = ['ZipCode','NumberOfReturns','ExemptionsRtn','DepedentExemptionsAmt','AGI','SalariesWagesRtn','SalariesWagesAmt', \
          'TaxableInterestRtn','TaxableInterestAmt', \
          'TotalTaxRtn','TotalTaxAmt','ScheduleCTotalRtn','ScheduleFTotalRtn',\
          'ScheduleARtn'
          ]
irsdata.columns= irsCol
irsdata.head(10)
chars_to_remove = ['*', '**', '--',',']
for col in irsdata.columns:
    irsdata[col] = irsdata[col].str.translate(None, ''.join(chars_to_remove))

zipcode = pd.read_csv('./DataPfiles/us_postal_codes.csv')
calzipcode = zipcode[zipcode.State == 'California']
# print irsdata.columns
# print calzipcode.columns
# print irsdata.ZipCode
irsdata['ZipCode'] = irsdata['ZipCode'].str.strip()  #strip white space in the cell
print sum(irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str)))  #match the rows in irsdata with calzipcode
irsdata2005 = irsdata[irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str))]
irsdata2005['AverageAGI'] = irsdata2005.AGI.astype(int) #/ irsdata2002['NumberOfReturns'])
irsdata2005['NumberOfReturns'] = irsdata2005['NumberOfReturns'].astype(int)
irsdata2005['AverageAGI'] = irsdata2005['AverageAGI'] * 1000 / irsdata2005.NumberOfReturns
print irsdata2005['AverageAGI'].mean()

irsdata2005full = pd.DataFrame(columns=irsdata2005.columns.to_series().append(pd.Series("Month")))

for x in range(1,irsdata2005.shape[0]):
    listtemp = []
    irsdata2005t = pd.DataFrame(columns=irsdata2005.columns)
    irsdata2005t = irsdata2005t.append([irsdata2005.iloc[x]]*12,ignore_index=True)
    for i in range(12):
        thisMonth = ("0%i"%(i+1,))[-2:]
        d = dateUtility.mkDateTime("2005-%s-02"%thisMonth)
        listtemp.append(dateUtility.mkLastOfMonth(d).strftime("%Y-%m-%d"))
    irsdata2005t["Month"] = pd.Series(listtemp)
    irsdata2005full = irsdata2005full.append(irsdata2005t, ignore_index=True)
    
irsdata2005full.set_index(['Month','ZipCode'],inplace=True)  #set dataframe index to match with zipcode price dataframe

### merge IRS data with zipcode price data
irsdatafull = irsdatafull.append(irsdata2005full)
dfPredictZipCodeFinal = pd.concat([dfPredictZipCode.unstack(level=1),irsdatafull.unstack(level=1)], axis=1)
dfPredictZipCodeFinal = dfPredictZipCodeFinal.stack(level=1)
print "done"

C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2396
55154.0374922
done


C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
### This cell is trying to combine the individual california zipcode data .p pickle file into a singe file
### so the main analytic code can load all zipcodes and their distance difference into a single data frame.

import os
import glob
import pandas as pd
from os import path
path = '.\\DataPfiles\\'

calzipcodeAllLong = pd.DataFrame()    
for filename in glob.glob(os.path.join(path, 'calzip*.p')):
    calzipcodeAllLong = pd.concat([calzipcodeAllLong, pd.read_pickle(filename)], axis=1)

dfname = './DataPfiles/' + 'calzipcodeAllLong' + ".p"
pickle.dump( calzipcodeAllLong, open( dfname, "wb" ) )
print 'Done: ' + dfname

In [ ]:
##Abnormally Detection method TWO
##Find out zipcode in df..InZipCode which has z score > 2 and in df..AcrossZipCode among the 10 closest zipcodes which has z score > 2



### This cell is trying to calculate the distance between two zipcodes (within California) in miles
import pandas as pd
import pickle
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
from math import radians, cos, sin, asin, sqrt
def calculateDistance(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    return km


geolocator = Nominatim()

zipcode = pd.read_csv('./DataPfiles/us_postal_codes.csv')
calzipcode = zipcode[zipcode.State == 'California']

startzipcodeIndex = int(100)    #last batch processed by script at test.ipynb
endzipcodeIndex = int(200)

for z in calzipcode.iloc[startzipcodeIndex:endzipcodeIndex,0]:
# for z in calzipcode.iloc[2:3,0]:
# for z in calzipcode['Postal Code']:    
    for i,r in calzipcode.iterrows():
        ziplon1 = calzipcode[calzipcode['Postal Code'] == int(z)]['Longitude']
        ziplat1 = calzipcode[calzipcode['Postal Code'] == int(z)]['Latitude']
        ziplon2 = r['Longitude']
        ziplat2 = r['Latitude']
        calzipcode.loc[i,z] = calculateDistance(ziplon1,ziplat1,ziplon2,ziplat2)

dfname = './DataPfiles/' + 'calzip' + str(int(calzipcode.iloc[startzipcodeIndex]['Postal Code'])) + "_to_" + str(int(calzipcode.iloc[endzipcodeIndex-1]['Postal Code'])) + ".p"
pickle.dump( calzipcode, open( dfname, "wb" ) )
print 'Done: ' + dfname
 
    
# geolocator = Nominatim()
# location = geolocator.geocode("94582")
# print(location.address)
# print((location.latitude, location.longitude))
# print(location.raw)

# # >> newport_ri = (41.49008, -71.312796)
# # >>> cleveland_oh = (41.499498, -81.695391)
# print(vincenty((geolocator.geocode("94582").latitude,geolocator.geocode("94582").longitude), (geolocator.geocode("94122").latitude,geolocator.geocode("94122").longitude)).miles)

In [1]:
### This cell as well as the cell after this one are just my playground to play with different functions.  This is not part of the project. 


from pylab import plot, ylim, xlim, show, xlabel, ylabel
from numpy import linspace, loadtxt
import numpy as np

r=3.0

x = p.head()
y = pz

def movingaverage(interval, window_size):
    window = np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

# plot(x,y)
# xlim(0,1000)

x_av = movingaverage(x, r)
# plot(x_av, y)

# xlabel("Months since Jan 1749.")
# ylabel("No. of Sun spots")
# show()
print x_av

p = df11.iloc[:,0]  #ALl Homes price
pz = df11.iloc[:,-1] 
print pz


t1 = df.iloc[:,1] 
t2 = df3.iloc[:,0]
print "mean is % 4.3F and sd is % 4.3F " % (t1.mean(),t1.std
                                            
                                            # add a new column for moving average of All Homes price


p = df11.iloc[:,0] #ALl Homes price
pz = df11.iloc[:,-1] 


window_size = 3.0 #set the number of sample to gathering centered in the middle
movingave = lambda x: np.convolve(x, np.ones(int(window_size))/float(window_size), 'same')
transformed = df11.groupby('ZipCode')
transformed['All Homes'].transform(movingave)
# df11.info()

# df11 = df11[df11['All Homes'].isnull()]
grouped = df11[['All Homes','ZipCode']].groupby('ZipCode')
b = pd.DataFrame()
newdf = pd.DataFrame()
for name,group in grouped:
    g = group.copy()
#     print group.shape
#     print "size: % 3.2F" % movingaverage(group['All Homes'],3).size
#     if sum(g.isnull()) < 0:
#         g['Moving Ave'] = movingaverage(group['All Homes'],3)

#     print g.shape
#     print g.head()

    #     if(newdf.isnull):
#         newdf = g
#     newdf = newdf.append(g)
#     b = group['All Homes']
#     a = movingaverage(group['All Homes'],3)
    
#     group["MovingAve"] = np.convolve(group['All Homes'], np.ones(int(window_size))/float(window_size), 'same')
#     newdf = newdf.append(group)
# print b
# newdf
# newdf.info()
    
    


# df11['Moving Ave'] = movingaverage(df11.iloc[:,0], 3)  #para#1 is All Homes price; para#2 is numer of sample to gather centered in the middle

# df11.head()
x                                             

SyntaxError: invalid syntax (<ipython-input-1-2f2d3327462f>, line 40)

In [21]:
np.average(dfAllHomes['93063'][(-1-y):-1])

483400.0

In [42]:
import pandas as pd
#dfAllHomesDiffFromMovAve

In [43]:
print dfAllHomesStdDevAcrossZipCode.shape
# dfAllHomesStdDevAcrossZipCode
# np.sum(dfAllHomesStdDevAcrossZipCode > 2)

(235, 1759)


In [44]:
#dfAllHomesDiffFromMovAve.mean(axis =1 )

In [1]:
dfAllHomesDiffFromMovAve.std(axis = 1)

NameError: name 'dfAllHomesDiffFromMovAve' is not defined

In [1]:
from geopy.geocoders import Nominatim
from geopy.distance import vincenty

geolocator = Nominatim()
location = geolocator.geocode("94582")
print(location.address)
print((location.latitude, location.longitude))
print(location.raw)

# >> newport_ri = (41.49008, -71.312796)
# >>> cleveland_oh = (41.499498, -81.695391)
print(vincenty((geolocator.geocode("94582").latitude,geolocator.geocode("94582").longitude), (geolocator.geocode("94122").latitude,geolocator.geocode("94122").longitude)).miles)

San Ramon, Contra Costa County, California, 94582, United States of America
(37.7608662693782, -121.903546728611)
{u'display_name': u'San Ramon, Contra Costa County, California, 94582, United States of America', u'importance': 0.225, u'place_id': u'128955624', u'lon': u'-121.903546728611', u'lat': u'37.7608662693782', u'licence': u'Data \xa9 OpenStreetMap contributors, ODbL 1.0. http://www.openstreetmap.org/copyright', u'boundingbox': [u'37.760816269378', u'37.760916269378', u'-121.90359672861', u'-121.90349672861'], u'type': u'postcode', u'class': u'place'}
31.8138952547


In [45]:
import pandas as pd
import pickle

df = pickle.load( open( ".\DataPfiles\calzip90001_to_90010.p", "rb" ) ) 

# print "abcde"
# zipcode = pd.read_csv('./DataPfiles/us_postal_codes.csv')
# calzipcode = zipcode[zipcode.State == 'California']

In [137]:
import os
from os import path
files = [f for f in os.listdir(".\DataPfiles\\")]# if path.isfile(f)]
files
# print os.listdir(".\DataPfiles\\")

['90621_to_91024.p',
 '93063_to_93524.p',
 'calzip90001_to_90010.p',
 'calzip90002_to_90001.p',
 'calzip90002_to_90005.p',
 'calzip90003_to_90003.p',
 'calzip90011_to_90020.p',
 'calzip90021_to_90050.p',
 'calzip90051_to_90174.p',
 'calzip90185_to_90620.p',
 'calzip90621_to_90621.p',
 'calzip90622_to_90638.p',
 'calzip90639_to_90746.p',
 'calzip90747_to_90848.p',
 'calzip90853_to_91024.p',
 'us_postal_codes.csv']

In [4]:
PtoR = dfPriceToRent.mean()
print dfPriceToRent.mean(),dfPriceToRent.std()
print PtoR.mean(),PtoR.std()

highPtoRzip = pd.DataFrame(dfPriceToRent.mean() > 18)
highPtoRzip[highPtoRzip[0] == True]
# targetZipCodes[targetZipCodes[0] == True]

ZipCode
90001    10.568421
90002     9.980175
90003    10.541053
90004    22.002281
90005    17.624035
90006    15.678772
90007    14.658772
90008    15.648246
90010    16.904912
90011    10.790526
90012    14.107719
90013    16.126842
90014    13.615439
90015    16.464211
90016    13.940175
90017    16.246491
90018    13.547719
90019    20.127018
90020    15.213860
90021          NaN
90022    13.211404
90023    12.158947
90024    18.184912
90025    17.881228
90026    19.939474
90027    23.031930
90028    17.345263
90029    18.350351
90031    14.681404
90032    14.548772
           ...    
96119    15.990000
96120    22.200000
96121    22.560000
96122    22.260000
96123    21.950000
96124    20.320000
96125    15.050000
96126    21.150000
96127    23.170000
96128    18.760000
96129    17.520000
96130     8.409167
96133    13.090000
96134     5.387000
96135    15.560000
96136          NaN
96137    15.302459
96140    22.848667
96141    25.432667
96142    21.357167
96143    18.330333
9614

,0
ZipCode,
90004,True
90019,True
90024,True
90026,True
90027,True
90029,True
90034,True
90035,True
90036,True


In [43]:
# dfPredictZipCodeFinal = pd.concat([dfPredictZipCodeFinal.unstack(level=1),irsdata2001full.unstack(level=1)], axis=1)
# dfPredictZipCodeFinal = dfPredictZipCodeFinal.stack(level=1)

# b = dfPredictZipCodeFinal.index.get_level_values(0) == '1998-09-30'  #for testing the result
# # b1 = dfPredictZipCodeFinal.index.get_level_values(0) == '2001-09-30'  #for testing the result
# dfPredictZipCodeFinal[b]  #for testing the result

# irsdatafull = irsdata1998full
# irsdatafull = irsdatafull.append(irsdata2001full)
# irsdatafull = irsdatafull.append(irsdata2002full)
# dfPredictZipCodeFinal = pd.concat([dfPredictZipCode.unstack(level=1),irsdatafull.unstack(level=1)], axis=1)
# dfPredictZipCodeFinal = dfPredictZipCodeFinal.stack(level=1)
# dfPredictZipCodeFinal

b = dfPredictZipCodeFinal.index.get_level_values(0) == '1998-09-30'  #for testing the result
b1 = dfPredictZipCodeFinal.index.get_level_values(0) == '2001-09-30'  #for testing the result
b2 = dfPredictZipCodeFinal.index.get_level_values(0) == '2002-09-30'  #for testing the result
b4 = dfPredictZipCodeFinal.index.get_level_values(0) == '2004-09-30'  #for testing the result
b5 = dfPredictZipCodeFinal.index.get_level_values(0) == '2005-09-30'  #for testing the result
dfPredictZipCodeFinal[b5]  #for testing the result

1 Bedroom  2 Bedroom  3 Bedroom  4 Bedroom  \
           ZipCode                                               
2005-09-30 90001       264000     309200     334100     354500   
           90002       261400     302100     319800     350100   
           90003       278700     321200     336600     359500   
           90004       501000     646600     905800    1321200   
           90005          NaN     533300     688500    1183600   
           90006          NaN     462300     542200     658900   
           90007          NaN     415700     465600     553600   
           90008       286200     507200     582900     696000   
           90009          NaN        NaN        NaN        NaN   
           90010          NaN     520300        NaN        NaN   
           90011          NaN     332400     350100     362100   
           90012          NaN     407100     445100        NaN   
           90013          NaN     453700        NaN        NaN   
           90014       354900        NaN        NaN        NaN   
           90015       425300     570200        NaN        NaN   
           90016       324300     431700     519700     581400   
           90017       380700        NaN        NaN        NaN   
           90018          NaN     426500     466300     536400   
           90019          NaN     640900     793900     863600   
           90020          NaN     424300     635600    2046600   
           90021          NaN        NaN        NaN        NaN   
           90022       296500     368200     404300     435900   
           90023          NaN     332600     360400     383800   
           90024          NaN     729000    1409600    2098500   
           90025       456200     696000     906200    1156900   
           90026       442000     545500     633400     731200   
           90027       445600     703700     994300    1519500   
           90028       521900     536400     674100        NaN   
           90029          NaN     587100     641800        NaN   
           90030          NaN        NaN        NaN        NaN   
...                       ...        ...        ...        ...   
           96124          NaN        NaN        NaN        NaN   
           96125          NaN        NaN        NaN        NaN   
           96126          NaN        NaN        NaN        NaN   
           96127          NaN        NaN        NaN        NaN   
           96128          NaN        NaN        NaN        NaN   
           96129          NaN        NaN        NaN        NaN   
           96130          NaN     142600     195800     250800   
           96132          NaN        NaN        NaN        NaN   
           96133          NaN        NaN        NaN        NaN   
           96134          NaN        NaN        NaN        NaN   
           96135          NaN        NaN        NaN        NaN   
           96136          NaN        NaN        NaN        NaN   
           96137          NaN        NaN     503200     731000   
           96140          NaN     533500     639200     783500   
           96141          NaN     463700     588400     822600   
           96142          NaN        NaN     577200     668200   
           96143       347000     402900        NaN     714900   
           96145          NaN     488800     646200     885000   
           96146          NaN        NaN     673800    1112900   
           96148          NaN        NaN     655700     837800   
           96150       296700     350700     468300     631300   
           96151          NaN        NaN        NaN        NaN   
           96152          NaN        NaN        NaN        NaN   
           96154          NaN        NaN        NaN        NaN   
           96155          NaN        NaN        NaN        NaN   
           96156          NaN        NaN        NaN        NaN   
           96158          NaN        NaN        NaN        NaN   
           96160          NaN        NaN        NaN        N